In [6]:
import pandas as pd
from tipo_stats import TipoStats

def get_target(max_value_item, valor_item, tipo_stats):
    '''
    Função para calcular o valor que sera usado para determinar quanto de stats sera necessario para chegar no maximo valor do item
    ex: max_value por ser lv 150 e spd é 15, valor_item é 10, entao eu preciso de 9(target). 9/2 = 4. O que somado com o valor do item 10 + 4 = 14. 1 a menos do que o max_value
    '''
    if tipo_stats in [TipoStats.MDR, TipoStats.PDR, TipoStats.CritChance, 
                    TipoStats.AddHit, TipoStats.AddShot, TipoStats.Parry, 
                    TipoStats.MagicPower]:
        target = ((max_value_item - valor_item) * 2) - 0.01
    else:
        target = ((max_value_item - valor_item) * 2) - 1
    
    print("Valor item:", valor_item)
    print("Target:", target)

    return target

# Função para ler os valores de um arquivo txt
def ler_valores(nome_arquivo):
    with open(nome_arquivo, 'r') as file:
        valores = [float(line.strip()) for line in file.readlines()]
    return valores

def get_max_value_item(item_lv, tipo_stats: TipoStats):
    '''
    Obtém o valor máximo do item de acordo com seu tipo de status e item lv.
    Ex: item do tipo SPD e lv 150, seu max_value é 15.
    '''
    df = pd.read_csv("docs/stats.csv")
    max_value_item = df[df["Item Lv"] == item_lv][tipo_stats.value].iloc[0]
    
    try:
        # Converte para string e substitui vírgula por ponto (caso exista)
        value_str = str(max_value_item).replace(',', '.')
        
        if tipo_stats in [TipoStats.MDR, TipoStats.PDR, TipoStats.CritChance, 
                          TipoStats.AddHit, TipoStats.AddShot, TipoStats.Parry, 
                          TipoStats.MagicPower]:
            max_value_item = float(value_str)  # Converte para float (aceita "20", "20.5", "30,5" -> 30.5)
        else:
            # Se for um tipo que deve ser int, converte para float primeiro (para evitar erro se for "30.5")
            max_value_item = int(float(value_str))  # Converte para int (arredonda para baixo)
    except (ValueError, TypeError) as e:
        print(f"Erro ao converter {tipo_stats}: {max_value_item} - {e}")
        max_value_item = 0  # Valor padrão em caso de erro
    
    print("Max value item:", max_value_item)
    return max_value_item

from itertools import combinations

def encontrar_melhor_combinacao_int_values(valores, target):
    '''
    Qual é a melhor combinação que somada seja o mais proximo possivel do target, segundo a lista de valores passada?
    Caso a todas as somas possiveis ultrapassem o target, trazer o valor mais proximo sem soma-los.
    Valor nao pode exceder o target
    '''
    # Melhor combinação encontrada
    melhor_soma = 0
    melhor_combinacao = []

    # Testa todas as combinações possíveis
    for r in range(1, len(valores) + 1):
        for combinacao in combinations(valores, r):

            soma_atual = sum([int(valor / 2) * 2 for valor in combinacao])

            if soma_atual <= target and soma_atual > melhor_soma:
                melhor_soma = soma_atual
                melhor_combinacao = combinacao

                melhor_soma_real = sum([int(valor / 2) for valor in melhor_combinacao])

    return melhor_combinacao, melhor_soma, melhor_soma_real

def encontrar_melhor_combinacao_geral(valores, target):
    '''
    Qual é a melhor combinação que somada seja o mais proximo possivel do target, segundo a lista de valores passada?
    Caso a todas as somas possiveis ultrapassem o target, trazer o valor mais proximo sem soma-los.
    Valor nao pode exceder o target
    '''
    # Melhor combinação encontrada
    melhor_soma = 0
    melhor_combinacao = []

    # Testa todas as combinações possíveis
    for r in range(1, len(valores) + 1):
        for combinacao in combinations(valores, r):
            soma_atual = sum(combinacao)
            if soma_atual <= target and soma_atual > melhor_soma:
                melhor_soma = soma_atual
                melhor_combinacao = combinacao

    return melhor_combinacao, melhor_soma
def encontrar_melhor_combinacao_bonus_dmg(valores, target):
    '''
    Qual é a melhor combinação que somada seja o mais proximo possivel do target, segundo a lista de valores passada?
    Caso a todas as somas possiveis ultrapassem o target, trazer o valor mais proximo sem soma-los.
    Valor nao pode exceder o target
    '''
    # Melhor combinação encontrada
    melhor_soma = 0
    melhor_combinacao = []

    valores = [valor-10 for valor in valores]

    # Testa todas as combinações possíveis
    for r in range(1, len(valores) + 1):
        for combinacao in combinations(valores, r):
            soma_atual = sum(combinacao)
            if soma_atual <= target and soma_atual > melhor_soma:
                melhor_soma = soma_atual
                melhor_combinacao = combinacao
    

    melhor_combinacao = [comb+10 for comb in melhor_combinacao]

    return melhor_combinacao, melhor_soma
# Função para encontrar a melhor combinação
def encontrar_melhor_combinacao_valores_subsequentes(valores, target, tipo_stats):
    '''
    Tem algum dos valores da lista que somando ele + (outro valor / 2) da o target ou um valor aproximado? A soma não pode exceder o target.
    '''
    melhor_combinacao = None
    melhor_diferenca = float('inf')

    if tipo_stats == TipoStats.BonusDmg:
        valores = [valor-10 for valor in valores]

    for v1 in valores:
        for v2 in valores:
            if v1 != v2:  # Evitar usar o mesmo valor para dividir por 2
                soma = v1 + (v2 / 2)
                if soma <= target:
                    diferenca = target - soma
                    if diferenca < melhor_diferenca:
                        melhor_diferenca = diferenca
                        melhor_combinacao = (v1, v2, soma)

    if melhor_combinacao and tipo_stats == TipoStats.BonusDmg:
        melhor_combinacao = [comb+10 for comb in melhor_combinacao]
        
    print(f'Melhor combinação: {melhor_combinacao}')
    
    return melhor_combinacao

In [7]:
#Parametros
item_lv = 150
tipo_stats = TipoStats.MDR
valor_item = 9.79

In [8]:
max_value_item = get_max_value_item(item_lv, tipo_stats)

target = get_target(max_value_item, valor_item, tipo_stats)

# Ler valores do arquivo
filename = f'docs/{tipo_stats.value}.txt'  # Ajuste o caminho conforme necessário
lista_valores_arquivo = ler_valores(filename)
#Remover valor_item da lista, já que será usado um item da propria lista
if valor_item in lista_valores_arquivo:
    lista_valores_arquivo.remove(valor_item)

Max value item: 22.5
Valor item: 9.79
Target: 25.41


In [9]:
if tipo_stats in [TipoStats.Stats, TipoStats.Luck, TipoStats.Spd, TipoStats.Skills]:
    melhor_combinacao, melhor_soma, melhor_soma_real = encontrar_melhor_combinacao_int_values(lista_valores_arquivo, target)
    
    valor_item_final = valor_item + melhor_soma_real

    print("Melhor combinação:", melhor_combinacao)
    print("Combinação Ótima:", max_value_item == valor_item_final + 1)
    print("Valor Item Final:", valor_item_final)
    print("Diferença Max Value e Valor Final:", max_value_item - valor_item_final)

elif tipo_stats == TipoStats.BonusDmg:
    melhor_combinacao, melhor_soma = encontrar_melhor_combinacao_bonus_dmg(lista_valores_arquivo, target)
    
    print("Melhor combinação:", melhor_combinacao)
    print(f"Soma mais próxima de {target}:", melhor_soma)
    print("Diferença:", target - melhor_soma)
    
else:
    melhor_combinacao, melhor_soma = encontrar_melhor_combinacao_geral(lista_valores_arquivo, target)

    print("Melhor combinação:", melhor_combinacao)
    print(f"Soma mais próxima de {target}:", melhor_soma)
    print("Diferença:", target - melhor_soma)

Melhor combinação: (10.73, 14.65)
Soma mais próxima de 25.41: 25.380000000000003
Diferença: 0.029999999999997584


In [10]:
melhor_combinacao = encontrar_melhor_combinacao_valores_subsequentes(lista_valores_arquivo, target, tipo_stats)

Melhor combinação: (18.24, 14.28, 25.38)
